In [1]:
from dash import Dash, dcc, html, Input, Output, State, ctx, dash_table
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta

# Generate the datasets
def generate_data():
    # Generate influencers data
    influencers = pd.DataFrame({
        'id': range(1, 101),
        'name': [f'Influencer_{i}' for i in range(1, 101)],
        'category': np.random.choice(['Fitness', 'Nutrition', 'Lifestyle', 'Wellness'], 100),
        'gender': np.random.choice(['Male', 'Female', 'Non-binary'], 100, p=[0.45, 0.45, 0.1]),
        'follower_count': np.random.randint(5000, 500000, 100),
        'platform': np.random.choice(['Instagram', 'YouTube', 'Twitter', 'Facebook'], 100)
    })

    # Generate posts data
    posts = pd.DataFrame({
        'influencer_id': np.random.choice(influencers['id'], 500),
        'platform': np.random.choice(['Instagram', 'YouTube', 'Twitter'], 500),
        'date': [datetime.today() - timedelta(days=np.random.randint(1, 60)) for _ in range(500)],
        'url': [f'https://social.com/post/{i}' for i in range(1, 501)],
        'caption': ['Check out this amazing product!' for _ in range(500)],
        'reach': np.random.randint(1000, 500000, 500),
    })
    posts['likes'] = posts['reach'] * np.random.uniform(0.01, 0.05, 500)
    posts['comments'] = posts['reach'] * np.random.uniform(0.001, 0.005, 500)

    # Generate tracking data
    tracking_data = pd.DataFrame({
        'source': ['influencer'] * 2000,
        'campaign': [f'Campaign_{np.random.randint(1, 11)}' for _ in range(2000)],
        'influencer_id': np.random.choice(influencers['id'], 2000),
        'user_id': [f'USER_{np.random.randint(1000, 9999)}' for _ in range(2000)],
        'brand': np.random.choice(['MuscleBlaze', 'HKVitals', 'Gritzo'], 2000),
        'product': [f'Product_{np.random.randint(1, 20)}' for _ in range(2000)],
        'date': [datetime.today() - timedelta(days=np.random.randint(1, 30)) for _ in range(2000)],
        'orders': np.random.randint(1, 5, 2000),
        'revenue': np.random.uniform(100, 5000, 2000)
    })

    # Generate payouts data
    payouts = pd.DataFrame({
        'influencer_id': np.random.choice(influencers['id'], 300),
        'basis': np.random.choice(['post', 'order'], 300, p=[0.7, 0.3]),
        'rate': np.random.uniform(100, 5000, 300),
        'orders': np.random.randint(1, 10, 300),
    })
    payouts['total_payout'] = np.where(payouts['basis'] == 'post', payouts['rate'], payouts['rate'] * payouts['orders'])
    
    return influencers, posts, tracking_data, payouts

# Generate the data
influencers, posts, tracking_data, payouts = generate_data()

app = Dash(__name__)
app.title = "HealthKart Influencer Campaign Dashboard"

app.layout = html.Div([
    html.H1("HealthKart Influencer Campaign Dashboard"),
    
    html.Div([
        html.H4("Using randomly generated data"),
        html.Button('Refresh Data', id='refresh-button'),
    ], style={'margin': '20px'}),

    html.Div(id='filter-ui'),
    html.Div(id='dashboard-output')
])

@app.callback(
    Output('filter-ui', 'children'),
    Input('refresh-button', 'n_clicks')
)
def display_filters(n_clicks):
    return html.Div([
        html.Label("Select Brand"),
        dcc.Dropdown(tracking_data['brand'].unique(), id='brand-filter', multi=True),

        html.Label("Select Platform"),
        dcc.Dropdown(influencers['platform'].unique(), id='platform-filter', multi=True),

        html.Label("Select Gender"),
        dcc.Dropdown(influencers['gender'].unique(), id='gender-filter', multi=True),

        html.Br(),
        html.Button('Generate Insights', id='generate-button')
    ])

@app.callback(
    Output('dashboard-output', 'children'),
    Input('generate-button', 'n_clicks'),
    Input('refresh-button', 'n_clicks'),
    State('brand-filter', 'value'),
    State('platform-filter', 'value'),
    State('gender-filter', 'value')
)
def update_dashboard(gen_clicks, refresh_clicks, brand_filter, platform_filter, gender_filter):
    if not gen_clicks and not refresh_clicks:
        return None
    
    # Regenerate data if refresh button was clicked
    if ctx.triggered_id == 'refresh-button':
        global influencers, posts, tracking_data, payouts
        influencers, posts, tracking_data, payouts = generate_data()
    
    df_inf = influencers.copy()
    df_posts = posts.copy()
    df_track = tracking_data.copy()
    df_pay = payouts.copy()

    if brand_filter:
        df_track = df_track[df_track['brand'].isin(brand_filter)]
    if platform_filter:
        df_inf = df_inf[df_inf['platform'].isin(platform_filter)]
        df_posts = df_posts[df_posts['platform'].isin(platform_filter)]
    if gender_filter:
        df_inf = df_inf[df_inf['gender'].isin(gender_filter)]

    merged = df_track.merge(df_inf, left_on='influencer_id', right_on='id', how='left')
    merged = merged.merge(df_pay, on='influencer_id', how='left')
    merged['total_revenue'] = merged.groupby('influencer_id')['revenue'].transform('sum')
    merged['total_payout'] = merged.groupby('influencer_id')['total_payout'].transform('first')
    merged['ROAS'] = merged['total_revenue'] / merged['total_payout']

    top_roas = merged.groupby(['influencer_id', 'name'])[['total_revenue', 'total_payout']].first().reset_index()
    top_roas['ROAS'] = top_roas['total_revenue'] / top_roas['total_payout']
    top_10 = top_roas.sort_values(by='ROAS', ascending=False).head(10)

    perf = merged.groupby(['category', 'gender'])['ROAS'].mean().reset_index()

    posts_summary = df_posts.groupby('influencer_id')[['reach']].mean().reset_index()

    poor = top_roas[top_roas['ROAS'] < 1.0]

    # Create some visualizations
    roas_fig = px.bar(top_10, x='name', y='ROAS', title='Top 10 Influencers by ROAS')
    perf_fig = px.bar(perf, x='category', y='ROAS', color='gender', barmode='group', title='Performance by Category and Gender')
    
    return html.Div([
        html.H4("Top Influencers by ROAS"),
        dcc.Graph(figure=roas_fig),
        dash_table.DataTable(
            top_10.to_dict('records'), 
            columns=[{"name": i, "id": i} for i in top_10.columns],
            style_table={'overflowX': 'auto'},
            page_size=10
        ),

        html.H4("Influencer Performance Summary"),
        dcc.Graph(figure=perf_fig),
        dash_table.DataTable(
            perf.to_dict('records'), 
            columns=[{"name": i, "id": i} for i in perf.columns],
            style_table={'overflowX': 'auto'},
            page_size=10
        ),

        html.H4("Posts Performance"),
        dash_table.DataTable(
            posts_summary.head(10).to_dict('records'), 
            columns=[{"name": i, "id": i} for i in posts_summary.columns],
            style_table={'overflowX': 'auto'},
            page_size=10
        ),

        html.H4("Influencers with Poor ROIs (<1.0)"),
        dash_table.DataTable(
            poor.to_dict('records'), 
            columns=[{"name": i, "id": i} for i in poor.columns],
            style_table={'overflowX': 'auto'},
            page_size=10
        )
    ])

if __name__ == '__main__':
    app.run(debug=True)

In [2]:
from dash.exceptions import PreventUpdate
import io
import base64
from dash import no_update

# Add these buttons to your layout (near the Generate Insights button)
export_buttons = html.Div([
    html.Button("Export to CSV", id="export-csv-btn"),
    html.Button("Export to PDF", id="export-pdf-btn"),
    dcc.Download(id="download-data"),
    html.Div(id='export-status')
])

# Add this callback
@app.callback(
    [Output('download-data', 'data'),
     Output('export-status', 'children')],
    [Input('export-csv-btn', 'n_clicks'),
     Input('export-pdf-btn', 'n_clicks')],
    [State('dashboard-output', 'children')],
    prevent_initial_call=True
)
def export_data(csv_clicks, pdf_clicks, dashboard_content):
    if not ctx.triggered:
        raise PreventUpdate
        
    if csv_clicks:
        # Get your final dataframe (modify as needed)
        df = merged[['name', 'category', 'total_revenue', 'total_payout', 'ROAS']]
        return dcc.send_data_frame(df.to_csv, "influencer_performance.csv"), "CSV exported successfully!"
    
    if pdf_clicks:
        # For PDF, you might want to use a library like pdfkit
        # This requires additional setup (wkhtmltopdf)
        from weasyprint import HTML
        html_content = f"""
        <h1>HealthKart Influencer Report</h1>
        {dashboard_content}
        """
        pdf = HTML(string=html_content).write_pdf()
        return dcc.send_bytes(pdf, "report.pdf"), "PDF exported successfully!"
    
    return no_update, ""